In [1]:
import re
import string
import numpy as np
import pandas as pd
import scipy as sp
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import feature_extraction, ensemble, model_selection
import xgboost as xg
import validators as vld

In [2]:

# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [34]:
df_train = pd.read_csv('./train_preprocessed.csv')
df_train.sample(n=5)

,Unnamed: 0,id,keyword,location,text,clean_text,text_length,upper_text_factor,tags_count,punct_factor,ann_count,urls_count,stop_words_factor,clean_tokens_factor,positive_factor,target
710,710,1075,bleeding,The Great State of Texas,@Benjm1 @TourofUtah @B1Grego saw that pileup o...,benjm1 tourofutah b1grego saw pileup tv keep r...,65,0.107692,0,0.046154,3,0,0.133333,0.866667,0.142857,1
1057,1057,1576,bomb,"Chicago, IL",The MF Life is a vocal and lyrical bomb. Saw h...,mf life vocal lyrical bomb saw live summer ama...,113,0.168142,0,0.070796,1,0,0.290323,0.709677,0.441176,0
485,485,751,avalanche,guaravitas,we'll crash down like an avalanche,well crash like avalanche,29,0.000000,0,0.034483,0,0,0.428571,0.571429,0.133333,0
7194,7194,10350,weapons,Incognito,WOOOOOOO RT @GameRant: Call of Duty: Black Ops...,wooooooo rt gamerant call duty black ops espor...,116,0.267241,0,0.112069,1,2,0.041667,0.958333,0.435897,0
3952,3952,5663,floods,Adventist - Lesson Sabbath,Myanmar Flooding: Heavy monsoon rains during t...,myanmar flooding heavy monsoon rain month july...,117,0.085470,0,0.076923,0,1,0.227273,0.772727,0.722222,1


In [17]:
from nltk import word_tokenize, pos_tag, ne_chunk

def extract_location(text):
    if not text or text is np.nan:
        return ''
    chunks = ne_chunk(pos_tag(word_tokenize(text)))
    for chunk in chunks:
        if hasattr(chunk, 'label'):
            if chunk.label() == 'GPE':
                return ' '.join(c[0] for c in chunk)
    return ''

df_sample['location_clean'] = df_sample['location'].apply(lambda txt: extract_location(txt))
df_sample.head(n=30)


,id,keyword,location,text,target,location_clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,
75,109,accident,NaN,RT @SleepJunkies: Sleeping pills double your r...,0,
150,215,airplane%20accident,"New Mexico, USA",@mickinyman @TheAtlantic That or they might be...,0,New Mexico
225,320,annihilated,NaN,70 years since we annihilated 100000 people in...,1,
300,442,apocalypse,NaN,Shadow boxing the apocalypse,1,
375,538,army,"Burbank,CA",@AP what a violent country get the army involv...,1,Burbank
450,653,attack,Selena | Britney | Hilary,Demi stans really think Heart Attack sold 5/6 ...,0,
525,759,avalanche,UK,Musician Kalle Mattson Recreates 34 Classic Al...,0,
600,868,bioterror,Across the Atlantic,#BreakingNews http://t.co/gAN14PW9TG FedEx no ...,0,
675,976,blaze,Gotham City,?? Yes I do have 2 guns ?? ??,0,Gotham City


## Vectorizing

In [35]:
count_vect = feature_extraction.text.TfidfVectorizer(max_features=2000)
train_vectors = count_vect.fit_transform(df_train['clean_text'])
train_vectors.shape

(7579, 2000)

In [36]:
count_vect.get_feature_names_out()[10:1000:10]

array(['70th', 'according', 'address', 'afternoon', 'air', 'album',
       'also', 'an', 'annual', 'apc', 'area', 'arsonist', 'attack',
       'avalanche', 'bag', 'based', 'beat', 'bell', 'bigger', 'blast',
       'blocked', 'boat', 'bound', 'bringing', 'brown', 'burn', 'buy',
       'calling', 'cancer', 'careful', 'cause', 'character', 'chile',
       'claim', 'clutch', 'collided', 'company', 'confirmed', 'cost',
       'covered', 'create', 'crushed', 'dad', 'data', 'dear', 'degree',
       'derail', 'destruction', 'different', 'displaced', 'download',
       'drinking', 'drowning', 'earlier', 'economy', 'em', 'engulfed',
       'eruption', 'event', 'exchange', 'exploded', 'facebook', 'famine',
       'faux', 'felt', 'finally', 'fit', 'flooding', 'football', 'found',
       'french', 'full', 'gang', 'germ', 'glass', 'good', 'green',
       'guess', 'hair', 'harm', 'health', 'hellfire', 'highway', 'hit',
       'horror', 'hundred', 'id', 'imagine', 'industry', 'insurer',
       'invoic

## Features selection

In [54]:
selected_features = [
    # 'text_length', 
    # 'urls_count',
    # 'stop_words_factor',
    # 'clean_tokens_factor',
    'positive_factor'
]
X_train = sp.sparse.hstack((train_vectors, df_train[selected_features]))

Y_train = df_train['target']
# X_train.columns = X_train.columns.astype(str)

X_train.shape, Y_train.shape

((7579, 2001), (7579,))

## Model selection

In [55]:
params = {
    'n_estimators': [100, 200],
    'max_depth': [100, None]
}
model = ensemble.RandomForestClassifier(n_jobs=2)
grid = model_selection.GridSearchCV(model, params, cv=5, n_jobs=2, scoring='f1', verbose=3)
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 2/5] END ...max_depth=100, n_estimators=100;, score=0.591 total time=   1.1s
[CV 1/5] END ...max_depth=100, n_estimators=100;, score=0.681 total time=   1.1s
[CV 4/5] END ...max_depth=100, n_estimators=100;, score=0.636 total time=   1.0s
[CV 3/5] END ...max_depth=100, n_estimators=100;, score=0.633 total time=   1.1s
[CV 5/5] END ...max_depth=100, n_estimators=100;, score=0.730 total time=   1.0s
[CV 1/5] END ...max_depth=100, n_estimators=200;, score=0.685 total time=   3.3s
[CV 2/5] END ...max_depth=100, n_estimators=200;, score=0.571 total time=   4.7s
[CV 3/5] END ...max_depth=100, n_estimators=200;, score=0.637 total time=   5.6s
[CV 4/5] END ...max_depth=100, n_estimators=200;, score=0.597 total time=   5.2s
[CV 5/5] END ...max_depth=100, n_estimators=200;, score=0.724 total time=   5.3s
[CV 1/5] END ..max_depth=None, n_estimators=100;, score=0.684 total time=   4.8s
[CV 2/5] END ..max_depth=None, n_estimators=100;,

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=2), n_jobs=2,
             param_grid={'max_depth': [100, None], 'n_estimators': [100, 200]},
             scoring='f1', verbose=3)

In [56]:
grid.best_params_, grid.best_score_

({'max_depth': 100, 'n_estimators': 100}, 0.654048570341332)

In [ ]:
params = {
    'learning_rate': [0.2, 0.3, 0.4],
    'n_estimators': [100, 130],
    'max_depth': [50, 70]
}
model = xg.XGBClassifier()
grid = model_selection.GridSearchCV(model, params, cv=5, n_jobs=-1, scoring='f1', verbose=3)
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END learning_rate=0.2, max_depth=50, n_estimators=100;, score=0.579 total time=  39.7s
[CV 3/5] END learning_rate=0.2, max_depth=50, n_estimators=100;, score=0.521 total time=  42.6s
[CV 2/5] END learning_rate=0.2, max_depth=50, n_estimators=100;, score=0.504 total time=  43.5s
[CV 5/5] END learning_rate=0.2, max_depth=50, n_estimators=100;, score=0.649 total time=  44.4s
[CV 4/5] END learning_rate=0.2, max_depth=50, n_estimators=100;, score=0.467 total time=  44.8s
[CV 3/5] END learning_rate=0.2, max_depth=50, n_estimators=130;, score=0.525 total time=  50.8s
[CV 1/5] END learning_rate=0.2, max_depth=50, n_estimators=130;, score=0.584 total time=  53.9s
[CV 2/5] END learning_rate=0.2, max_depth=50, n_estimators=130;, score=0.509 total time=  57.4s
[CV 5/5] END learning_rate=0.2, max_depth=50, n_estimators=130;, score=0.644 total time=  55.1s
[CV 1/5] END learning_rate=0.2, max_depth=70, n_estimators=100;, score=0.58

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.3, 0.4],
                         'max_depth': [50, 70], 'n_estimators': [100, 130]},
             scoring='f1', verbose=3)

In [ ]:
results = pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')[0:5].T
results

,3,5,4,2,6
mean_fit_time,64.570168,49.828529,42.180478,60.640564,50.84793
std_fit_time,5.610396,1.587964,1.764037,3.725105,0.910546
mean_score_time,1.285648,1.149032,1.27392,1.48139,1.259953
std_score_time,0.113405,0.106994,0.187176,0.247323,0.170656
param_learning_rate,0.2,0.3,0.3,0.2,0.3
param_max_depth,70,50,50,70,70
param_n_estimators,130,130,100,100,100
params,"{'learning_rate': 0.2, 'max_depth': 70, 'n_est...","{'learning_rate': 0.3, 'max_depth': 50, 'n_est...","{'learning_rate': 0.3, 'max_depth': 50, 'n_est...","{'learning_rate': 0.2, 'max_depth': 70, 'n_est...","{'learning_rate': 0.3, 'max_depth': 70, 'n_est..."
split0_test_score,0.594752,0.60554,0.590385,0.588008,0.605769
split1_test_score,0.519793,0.511709,0.511304,0.520994,0.505263


In [ ]:
grid.best_params_, grid.best_score_

({'learning_rate': 0.2, 'max_depth': 70, 'n_estimators': 130},
 0.5547965221998632)